In [148]:
import pandas as pd
import sqlite3
import re

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'collegeprofiles.db' AS colleges")

# print(cursor.execute("""
# SELECT * from colleges.university_profiles as c JOIN profiles as p on c.school=p.school limit 2
# """).fetchall())

def convert_class_rank(cr_str):
    cr = cr_str.split(' ')
    return int(cr[0])/int(cr[2])

def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if sat >= 2380:
        return 36
    if sat < 2380 and sat >= 2290:
        return 35
    if sat < 2290 and sat >= 2220:
        return 34
    if sat < 2220 and sat >= 2140:
        return 33
    if sat < 2140 and sat >= 2080:
        return 32
    if sat < 2080 and sat >= 2020:
        return 31
    if sat < 2020 and sat >= 1980:
        return 30
    if sat < 1980 and sat >= 1920:
        return 29
    if sat < 1920 and sat >= 1860:
        return 28
    if sat < 1860 and sat >= 1800:
        return 27
    if sat < 1800 and sat >= 1740:
        return 26
    if sat < 1740 and sat >= 1680:
        return 25
    if sat < 1680 and sat >= 1620:
        return 24
    if sat < 1620 and sat >= 1560:
        return 23
    if sat < 1560 and sat >= 1510:
        return 22
    if sat < 1510 and sat >= 1450:
        return 21
    if sat < 1450 and sat >= 1390:
        return 20
    if sat < 1390 and sat >= 1330:
        return 19
    if sat < 1330 and sat >= 1270:
        return 18
    if sat < 1270 and sat >= 1210:
        return 17
    if sat < 1210 and sat >= 1140:
        return 16
    if sat < 1140 and sat >= 1060:
        return 15
    if sat < 1060 and sat >= 990:
        return 14
    if sat < 990:
        return 13

df = pd.read_sql_query("SELECT * from colleges.university_profiles as c JOIN profiles as p on c.school=p.school", conn)
# df  = pd.read_sql_query("SELECT * from profiles", conn)

# CLEAN UNIVERSITY DATA
# drop sat averages
for col in ['sat_math', 'sat_reading', 'sat_writing']:
    df = df.drop(col, axis=1)
# remove duplicate school column
df = df.loc[:,~df.columns.duplicated()]
# handle nones in all columns
df.fillna(value=pd.np.nan, inplace=True)
# remove - in avg_act
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
# cost attendance in state vs out state
df['cost_attendance'] = df['cost_attendance'].map(lambda x: x[x.find("$")+1:x.find("<")-1] if not isinstance(x,float) else x)
# fix faculty total none as string bug
df["faculty_total"] = df["faculty_total"].map(lambda x : None if x == "None" else x)
# weird act nan bug
df = df[df.avg_act != "nan"]

# CLEAN INDIVIDUAL PROFILES
# sums profiles sat individual scores and drops them
df['sat_c'] = df['sat_m']+df['sat_r']+df['sat_w']
for col in ['sat_m', 'sat_r', 'sat_w']:
    df = df.drop(col, axis=1)
# converts all sat to act if act is not present
df['act'] = df.apply(lambda x: convert_sat_to_act(x['sat_c'] if pd.isnull(x['act']) else x['act']), axis=1)
# drop nan
df = df.dropna()
# drop class ranks that are not in "1 of 200" format
df = df[df.class_rank.str.match('\d* (of) \d*', na=False)]
# converts class rank to decimal
df['class_rank'] = df['class_rank'].apply(lambda x: convert_class_rank(x))
# convert string label to int
replace_columns = ['hs_type', 'gender', 'status', 'hs_state', 'school', 'institution_type']
df['status'].replace({'Deferred': 'Denied', 'Wait-Listed': 'Denied', 'Will Attend': 'Accepted'}, inplace=True)
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)


In [149]:
from sklearn.model_selection import train_test_split

X = df.drop('year', axis=1)
X = X.drop('status', axis=1)
X = X.drop('athlete', axis=1)
X = X.drop('sat_c', axis=1)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(X.applymap(is_nan))
    
y = df['status']
X_train, X_test, y_train, y_test = train_test_split(X, y)


       school  avg_gpa  avg_act  cost_attendance  average_freshman_aid  \
0       False    False    False            False                 False   
4       False    False    False            False                 False   
7       False    False    False            False                 False   
10      False    False    False            False                 False   
12      False    False    False            False                 False   
15      False    False    False            False                 False   
16      False    False    False            False                 False   
17      False    False    False            False                 False   
18      False    False    False            False                 False   
21      False    False    False            False                 False   
24      False    False    False            False                 False   
25      False    False    False            False                 False   
26      False    False    False       

In [150]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  """


In [151]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(8,8,8),max_iter=1000)


In [152]:
mlp.fit(X_train,y_train)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(8, 8, 8), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [153]:
predictions = mlp.predict(X_test)


In [154]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))


[[580  54]
 [ 61  95]]
              precision    recall  f1-score   support

           0       0.90      0.91      0.91       634
           1       0.64      0.61      0.62       156

   micro avg       0.85      0.85      0.85       790
   macro avg       0.77      0.76      0.77       790
weighted avg       0.85      0.85      0.85       790



In [155]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
score = logisticRegr.score(X_test, y_test)
print(score)

0.8734177215189873


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
